In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing libs......

In [2]:
from keras.models import load_model
from time import sleep
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import os
import tensorflow
import pandas as pd
import librosa
import soundfile
from IPython.display import Audio

In [3]:
from moviepy.editor import *

video = VideoFileClip("/content/drive/MyDrive/EE_769_project/Happy_final_video.mp4")

In [4]:
audio = video.audio
audio.write_audiofile("audio.mp3")

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


In [5]:
video_without_audio = video.without_audio()
video_without_audio.write_videofile("video_without_audio.mp4")

Moviepy - Building video video_without_audio.mp4.
Moviepy - Writing video video_without_audio.mp4



Moviepy - Done !
Moviepy - video ready video_without_audio.mp4


#Image Classification

In [ ]:
#!pip install tensorflow==2.11.0

In [6]:
model = load_model('/content/drive/MyDrive/EE_769_project/model.h5')

In [7]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [8]:
face_classifier = cv2.CascadeClassifier('/content/drive/MyDrive/EE_769_project/haarcascade_frontalface_default.xml')

In [9]:
cap = cv2.VideoCapture('/content/video_without_audio.mp4')

angry = []
disgust = []
fear = []
happy = []
neutral = []
sad = []
surprise = []

while True:
    _, frame = cap.read()

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
      roi_gray = gray[y:y+h,x:x+w]
      roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

    resized = cv2.resize(roi_gray, (48, 48))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 48, 48, 1))
    prediction = model.predict(reshaped)

    prediction = prediction.flatten()
    prediction = prediction.tolist()
    #print(prediction)

    angry.append(prediction[0])
    disgust.append(prediction[1])
    fear.append(prediction[2])
    happy.append(prediction[3])
    neutral.append(prediction[4])
    sad.append(prediction[5])
    surprise.append(prediction[6])

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 30ms/step


error: ignored

In [10]:
data = {'Angry':angry, 'Disgust':disgust, 'Fear':fear, 'Happy':happy, 'Neutral':neutral, 'Sad': sad, 'Surprise':surprise}
df = pd.DataFrame(data)

In [11]:
df.shape

(47, 7)

In [12]:
df.mean()

Angry       0.072381
Disgust     0.007507
Fear        0.111848
Happy       0.467495
Neutral     0.107636
Sad         0.102065
Surprise    0.131068
dtype: float64

#Speech Recognition

In [ ]:
#from keras.models import load_model
import joblib

model_2 = joblib.load('/content/drive/MyDrive/EE_769_project/model_speech_recog_updated.joblib')

In [ ]:
import librosa
import numpy as np

audio = '/content/drive/MyDrive/EE_769_project/TESS Toronto emotional speech set data/Neutral/OAF_came_neutral.wav'

To check whether the audio file is stereo or mono

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment

# Load the audio file
audio_test = AudioSegment.from_file(audio)

# Check if the audio has only one channel (mono)
if audio_test.channels == 1:
    print("The audio file is mono.")
else:
    print("The audio file is stereo.")

The audio file is mono.


Converting .mp3 to .wav

In [ ]:
from pydub import AudioSegment

audio_convert = AudioSegment.from_file(audio, format="mp3")
audio_convert = audio_convert.set_channels(2)  
audio_convert.export("stereo_audio.wav", format="wav")

<_io.BufferedRandom name='stereo_audio.wav'>

In [ ]:
stereo_audio = '/content/stereo_audio.wav'

In [ ]:
Audio(stereo_audio)

Spliting stereo audio into 2 mono audio

In [ ]:
audio_split = AudioSegment.from_file("stereo_audio.wav", format="wav")

# Split the audio file into two mono audio files
left_channel = audio_split.split_to_mono()[0]
right_channel = audio_split.split_to_mono()[1]

# Save the left and right channels as separate audio files
left_channel.export("stereo_left_channel.wav", format="wav")
right_channel.export("stereo_right_channel.wav", format="wav")

<_io.BufferedRandom name='stereo_right_channel.wav'>

In [ ]:
Audio('/content/stereo_left_channel.wav')

In [ ]:
Audio('/content/stereo_right_channel.wav')

Converting stereo to mono

In [ ]:
from pydub import AudioSegment

# Load the audio file
sound = AudioSegment.from_file(audio)

# Convert to mono channel
sound = sound.set_channels(1)

# Set the sample rate to 16000Hz
sound = sound.set_frame_rate(16000)

# Export the modified audio file
sound.export("output_file.wav", format="wav")

<_io.BufferedRandom name='output_file.wav'>

In [ ]:
mono_audio = '/content/output_file.wav'

In [ ]:
Audio(mono_audio)

In [ ]:
def extract_feature(file_name, **kwargs):
    
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")


    with soundfile.SoundFile(file_name) as sound_file:

        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])


        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
            
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))

        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))

        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))

        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))

    return result

In [ ]:
X_2 = []
features = extract_feature('/content/stereo_left_channel.wav', mfcc=True, chroma=True, mel=True)
X_2.append(features)
features = extract_feature('/content/stereo_right_channel.wav', mfcc=True, chroma=True, mel=True)
X_2.append(features)

In [ ]:
X_2_new = np.array(X_2)
X_2_new.shape

(1, 180)

In [ ]:
results = []
proba_predictions = model_2.predict_proba(X_2_new[0,:].reshape(1,-1))
results.append(proba_predictions.tolist())
proba_predictions = model_2.predict_proba(X_2_new[1,:].reshape(1,-1))
results.append(proba_predictions.tolist())
results = np.array(results)
results = results.flatten()
results = results.reshape(2,7)

In [ ]:
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Surprise', 'Sad']
df_2 = pd.DataFrame(results, columns = labels)

In [ ]:
df_2.mean()

0.00011862953037956697

In [ ]:
(df.mean()+df_2.mean())/2

Angry       0.036518
Disgust     0.003754
Fear        0.055505
Happy       0.231950
Neutral     0.054573
Sad         0.053106
Surprise    0.064652
dtype: float64